In [1]:
**Description:** Demonstrate how to make a new analysis tool, a plot/metric

**Contact authors:** Sophie Reed

**Last verified to run:** 2023-05-23

**LSST Science Piplines version:** w_2023_20

**Container Size:** Medium (or larger)

**Location:** This notebook points to files on the S3DF cluster at the USDF. Update paths accordingly if you are running elsewhere.

**Skills:** Make a new analysis tool and have a play with some data

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1830697800.py, line 5)

Import statements that we will need later

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.analysis.tools
from lsst.analysis.tools.interfaces import AnalysisTool
from lsst.analysis.tools.actions.vector import CoaddPlotFlagSelector, SnSelector, StarSelector, LoadVector, ExtinctionCorrectedMagDiff
from lsst.analysis.tools.actions.scalar import MedianAction, MeanAction, SigmaMadAction
from lsst.analysis.tools.actions.plot import SkyPlot
from lsst.analysis.tools.atools import ShapeSizeFractionalDiff

import lsst.daf.butler as dafButler
from lsst.analysis.tools.interfaces._task import _StandinPlotInfo

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

Now we have a tool we need some data to run it on. We're going to run it on the object table tract.

In [ ]:
dataId = {"skymap": "hsc_rings_v1", "tract": 9813, "instrument": "HSC"}
collections = ['u/bechtol']
repo = '/sdf/group/rubin/user/bechtol/bootcamp_2023_backup/rc2_subset/SMALL_HSC/'
Butler = dafButler.Butler(repo)
t = Butler.get("objectTable_tract", dataId=dataId, collections=collections)

We want to include some information on the plot for future reference, as much information as possible should be included here so that future you is less annoyed with past you when they can't recreate the plot.

If you can't be bothered to make a dict of the information to include then you can use ```_StandInPlotInfo``` from ```lsst.analysis.tools.interfaces._task```.

In [ ]:
plotInfoDict = {"bands": "i", "tract": 9813, "tableName": "objectTable_tract", "run": collections[0]}

Now we have all of the pieces let's run the tool! You need to remember to call finalize before trying to get results back.

A plot has been made, but it looks a bit sparse, only 333 objects that made it to the plot. Let's look into why, first step, let us look at each selector individually and see what they are doing to the input data.

It seems that the S/N selector is removing a lot of objects, typically we expect a threshold of 300 to be appropriate for HSC data. Let's investigate the S/N across the whole catalogue.

Let's check that the S/N issues aren't limited to just a couple of patches

So all the patches have some high S/N points on them let's look at an image and see if it looks like junk.

The image looks fine, next lets look at how many visits went into each point. ```i_inputCount``` tells you how many visits covered the object.

In [ ]:
def catOutline(ras, decs):
    """A utility function to help draw outlines of points of ra and dec
    
    Parameters
    ----------
    ras : iterable
        The R.A. of the points
    decs : iterable
        The declination of the points
        
    Returns
    -------
    xMids : np.array
        The midpoints of the bins on the x axis
    yMids : np.array
        The midpoints of the bins on the y axis
    im : np.array
        A 2D array of the number of points in each bin
    """
    
    xWalls = np.linspace(np.min(ras) - 0.1, np.max(ras) + 0.1, 101)
    yWalls = np.linspace(np.min(decs) - 0.1, np.max(decs) + 0.1, 101)
    
    im, xBins, yBins = np.histogram2d(ras, decs, bins=(xWalls, yWalls))
    xMids = 0.5*(xBins[:-1] + xBins[1:])
    yMids = 0.5*(yBins[:-1] + yBins[1:])
    im[im > 0] = 1
    
    return xMids, yMids, im
    

To look at the visits that went into our object table we need to first find the data that went into it. The coadds are made from the visit level calexps so we will have a look at the calexps that are available in the butler. This gives us a list of visits. Once we have the visits we can read in the tables for each visit and then use those to plot the overlaps.


There are only 8 visits and they don't all overlap everywhere, the data isn't going to have very high S/N.

Now we know that the data isn't very deep we can adjust the signal to noise threshold so that we have more points on the plot

The data still doesn't cover all the area that the visits covered, let's look at why.
We'll plot the patch outlines over the input visits, we know that only four patches are in the final catalogue.